## Neural networks modeling

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from utils.transform_scale import transform_scale_df, TARGET_VARIABLE_COLUMN

import torch
import torch.nn as nn
import torch.optim as optim

DATA_PATH = Path("data")

/Users/niharika/opt/anaconda3/envs/cs5228/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load augmented data
train_augmented = pd.read_csv(DATA_PATH / "train-augmented.csv", parse_dates=["month"])
test_augmented = pd.read_csv(DATA_PATH / "test-augmented.csv", parse_dates=["month"])

train_augmented.head()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,eco_category,lease_commence_date,latitude,...,mean_age_m,std_age_f,std_age_m,pri_sch_dist,pri_sch,sec_sch_dist,sec_sch,mall_dist,mrt_name,mrt_dist
0,2001-08-01,pasir ris,4 room,440,pasir ris drive 4,118.0,model a,uncategorized,1989,1.369008,...,36.167630,20.331631,19.999478,0.344087,Loyang Primary School,0.428301,Pasir Ris Crest Secondary School,1.033216,Pasir Ris,1.137522
1,2014-10-01,punggol,5 room,196B,punggol field,110.0,improved,uncategorized,2003,1.399007,...,31.967676,20.103889,19.793305,0.160852,Edgefield Primary School,0.312383,Meridian Secondary School,0.806040,Cove,0.118373
2,2020-09-01,sengkang,5 room,404A,fernvale lane,112.0,premium apartment,uncategorized,2004,1.388348,...,34.164736,20.311337,19.947820,0.184906,Fernvale Primary School,0.558380,Pei Hwa Secondary School,0.452556,Fernvale,0.481153
3,2000-10-01,clementi,3 room,375,clementi avenue 4,67.0,new generation,uncategorized,1980,1.318493,...,40.577282,21.625967,21.440329,0.304561,Pei Tong Primary School,0.619132,Clementi Town Secondary School,0.456499,Clementi,0.423320
4,2013-01-01,bukit batok,3 room,163,bukit batok street 11,73.0,model a,uncategorized,1985,1.348149,...,38.318241,20.497124,20.287059,0.233809,Princess Elizabeth Primary School,0.217911,Bukit Batok Secondary School,0.764172,Bukit Batok,0.774220


In [3]:
# See linear.ipynb for details - code copied from there
# Split the train data into train and test
X = train_augmented.drop(columns=TARGET_VARIABLE_COLUMN)
y = train_augmented[TARGET_VARIABLE_COLUMN]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Transform and scale the data
# See utils/transform_scale.py for details
X_train, X_test = transform_scale_df(X_train, X_test)
X_train.head()

,month,lease_commence_date,floor_area_sqm,elevation,median_storey,distance_to_IHL,distance_to_CR,distance_to_market_hawker,mean_age_f,mean_age_m,...,flat_type_3 room,flat_type_4 room,flat_type_5 room,flat_type_executive,flat_type_multi generation,region_central region,region_east region,region_north region,region_north-east region,region_west region
424394,24202,23700,-0.594695,2.295101,0.669352,-0.715163,0.121163,-0.737495,1.492759,1.477374,...,0,1,0,0,0,1,0,0,0,0
120565,24006,23832,1.113372,0.073680,-1.182206,0.259262,0.781332,0.410626,0.390655,0.458419,...,0,0,1,0,0,0,0,0,1,0
145559,24198,23640,-1.468591,0.496808,-1.182206,-0.994291,1.135453,-0.785118,1.419774,1.154660,...,1,0,0,0,0,1,0,0,0,0
132809,24108,23844,-0.952198,0.602590,0.052166,0.087701,-0.070879,1.643412,-0.081596,0.088005,...,1,0,0,0,0,0,0,0,0,1
32978,24110,23952,-0.515250,1.025718,-1.182206,1.390263,-1.183845,-0.081635,-1.181189,-1.145237,...,0,1,0,0,0,0,0,1,0,0


In [4]:
list(X_train.columns)

['month',
 'lease_commence_date',
 'floor_area_sqm',
 'elevation',
 'median_storey',
 'distance_to_IHL',
 'distance_to_CR',
 'distance_to_market_hawker',
 'mean_age_f',
 'mean_age_m',
 'std_age_f',
 'std_age_m',
 'pri_sch_dist',
 'sec_sch_dist',
 'mall_dist',
 'mrt_dist',
 'flat_type_1 room',
 'flat_type_2 room',
 'flat_type_3 room',
 'flat_type_4 room',
 'flat_type_5 room',
 'flat_type_executive',
 'flat_type_multi generation',
 'region_central region',
 'region_east region',
 'region_north region',
 'region_north-east region',
 'region_west region']

In [5]:
# Construct a basic fully connected feed forward neural network
class ANN(nn.Module):
    def __init__(self, input_size):
        super(ANN, self).__init__()
        self.input_size = input_size
        self.layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        return self.layers(x)


In [6]:
# We min/max scale the month/lease_commence_date as well
scaler = MinMaxScaler()
columns = ["month", "lease_commence_date"]

X_train[columns] = scaler.fit_transform(X_train[columns])
X_test[columns] = scaler.transform(X_test[columns])

X_train.head()


,month,lease_commence_date,floor_area_sqm,elevation,median_storey,distance_to_IHL,distance_to_CR,distance_to_market_hawker,mean_age_f,mean_age_m,...,flat_type_3 room,flat_type_4 room,flat_type_5 room,flat_type_executive,flat_type_multi generation,region_central region,region_east region,region_north region,region_north-east region,region_west region
424394,0.808,0.169811,-0.594695,2.295101,0.669352,-0.715163,0.121163,-0.737495,1.492759,1.477374,...,0,1,0,0,0,1,0,0,0,0
120565,0.024,0.377358,1.113372,0.073680,-1.182206,0.259262,0.781332,0.410626,0.390655,0.458419,...,0,0,1,0,0,0,0,0,1,0
145559,0.792,0.075472,-1.468591,0.496808,-1.182206,-0.994291,1.135453,-0.785118,1.419774,1.154660,...,1,0,0,0,0,1,0,0,0,0
132809,0.432,0.396226,-0.952198,0.602590,0.052166,0.087701,-0.070879,1.643412,-0.081596,0.088005,...,1,0,0,0,0,0,0,0,0,1
32978,0.440,0.566038,-0.515250,1.025718,-1.182206,1.390263,-1.183845,-0.081635,-1.181189,-1.145237,...,0,1,0,0,0,0,0,1,0,0


In [8]:
# Convert the data to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=True)

# Define the model
model = ANN(X_train_tensor.shape[1])

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 50


In [11]:
# Train the model
for epoch in range(epochs):
    running_loss = 0.0
    for X, y in train_loader:
        X = X
        y = y
        optimizer.zero_grad()
        output = model(X)
        output = output.squeeze()
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1} / {epochs}, Training loss: {epoch_loss:.4f}")



Epoch 1 / 50, Training loss: 2492348513.2800
Epoch 2 / 50, Training loss: 2368254532.0770
Epoch 3 / 50, Training loss: 2289018581.3333
Epoch 4 / 50, Training loss: 1872725634.1807
Epoch 5 / 50, Training loss: 793664388.3615
Epoch 6 / 50, Training loss: 675849101.2741
Epoch 7 / 50, Training loss: 625094506.6193
Epoch 8 / 50, Training loss: 592178170.0741
Epoch 9 / 50, Training loss: 575412103.8696
Epoch 10 / 50, Training loss: 556366873.9319
Epoch 11 / 50, Training loss: 539261120.8059
Epoch 12 / 50, Training loss: 526473470.8622
Epoch 13 / 50, Training loss: 510104759.9407
Epoch 14 / 50, Training loss: 496427204.0770
Epoch 15 / 50, Training loss: 494548604.1600
Epoch 16 / 50, Training loss: 483538094.2222
Epoch 17 / 50, Training loss: 473981825.6593
Epoch 18 / 50, Training loss: 465231306.3822
Epoch 19 / 50, Training loss: 464867738.4059
Epoch 20 / 50, Training loss: 451624592.2133
Epoch 21 / 50, Training loss: 452986676.2904
Epoch 22 / 50, Training loss: 444975667.4370
Epoch 23 / 50, 

In [12]:
# Evaluate the model
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_test_tensor, y_pred.squeeze())
    y_pred = y_pred.numpy()
    print(f"Test loss: {test_loss.item()}")
    print(f"Mean squared error: {mean_squared_error(y_test, y_pred)}")
    print(f"Mean absolute error: {mean_absolute_error(y_test, y_pred)}")
    print(f"R2 score: {r2_score(y_test, y_pred)}")

Test loss: 385760832.0
Mean squared error: 385760816.2295561
Mean absolute error: 14140.200317183862
R2 score: 0.9769702642754826
